In [2]:
import numpy as np
import os
import numpy as np
import matplotlib.pyplot as plt


from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply , Embedding
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

# from faker import Faker
# import random
# from tqdm import tqdm
# from babel.dates import format_date
# import matplotlib.pyplot as plt
# %matplotlib inline


Using TensorFlow backend.
/home/isro/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/isro/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/isro/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/isro/.local/lib/python3.6/site-packages/tensorflow/python/framewo

In [3]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

In [4]:
words, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [5]:
print(np.shape(word_to_vec_map["father"]))

(50,)


In [6]:
input_sentences=[]

filepath = 'inp_dat.txt'
with open(filepath) as fp:
    line = fp.readline()
    cnt = 1
    while cnt <3000 :
        line = fp.readline()
        if cnt >15:
            input_sentences.append(line)
        cnt+=1
print(input_sentences[0])

Yes, I'm coming.



In [7]:
def wordcleaner(word):
    temp=""
    for char in word:
#         print(char)
        if char!= "," and char!= "!" and char!= "?" and char!= "." and char!= "'" and char!= "-":
#         if char!=',':
            temp+=char
    return temp
wordcleaner("i'm")

'im'

In [8]:
post_emb_list=[]

for l in input_sentences:
#     print(l)
    temp=[]
    for word in l.lower().split():
#         print(word)
        word = wordcleaner(word)
#         if word!="":
        w = word_to_vec_map.get(word)
        temp.append(w)
#         print(np.shape(temp))
    post_emb_list.append(temp)
    

In [9]:
max_len=20

In [55]:
batch_size = 10
context = np.zeros((10,max_len))
response = np.zeros((10,max_len))


In [56]:
# add = np.zeros((1,50))
# for l in post_emb_list:
#     while np.shape(l)!=(10,):
#         l.append(add)
    
# print(np.shape(post_emb_list[3]))

In [57]:
vocab_size = 10000
embedding_dim = 100
max_length = 20
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(input_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(input_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type,padding="post")
print(padded[6])
print(np.shape(padded))

index_word = {}
for word , index in word_index.items():
    index_word[index] = word

[ 19 401 156   7 282  33 228 637  10  44  80   0   0   0   0   0   0   0
   0   0]
(2984, 20)


In [58]:
index=0
for idx in range(0,20,2):
    context[index,:] = padded[idx]
    response[index,:] = padded[idx+1]
    index +=1
    
print(np.shape(context))
print(context[6])
print(response[0])
response = list(response.swapaxes(0,1))


(10, 20)
[143. 403.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.]
[ 12. 343.  19. 518. 519.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.]


In [59]:
import os
embeddings_index = {}
f = open( 'glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [60]:
embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [61]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            50,
                            weights=[embedding_matrix],
                            input_length=20,
                            trainable=False)

In [62]:
Tx = 20
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(20, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)


In [63]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = concatenator(inputs =([a,s_prev]))
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas,a])
    ### END CODE HERE ###
    
    return context


In [64]:
def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

In [65]:
post_activation_LSTM_cell = LSTM(128, return_state = True) # post-attention LSTM 
output_layer = Dense(20000, activation=softmax)

In [66]:

def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):

    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx,))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    e = embedding_layer(X)
    a = Bidirectional(LSTM(units = n_a ,return_sequences = True))(e)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(inputs = context,initial_state = [s,c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs = [X,s0,c0],outputs = outputs)
    
    ### END CODE HERE ###
    
    return model

In [69]:
model = model(20, 20,128, 128, 20000, 20000)


In [70]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 20, 50)       68250       input_3[0][0]                    
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 20, 256)      183296      embedding_2[0][0]                
____________________________________________________________________________________________

In [71]:
opt = Adam(lr = 0.005, beta_1 = 0.9,beta_2 = 0.999 , decay = 0.01)
model.compile(optimizer = opt , loss = "sparse_categorical_crossentropy" ,metrics = ['accuracy'])

In [72]:
s0 = np.zeros((10, 128))
c0 = np.zeros((10, 128))

In [73]:
model.fit([context,s0,c0],response,epochs=10,batch_size=1)

Epoch 1/10
10/10 [==============================] - 8s 812ms/step - loss: 144.7767 - dense_9_loss: 6.2808 - dense_9_accuracy: 0.0000e+00 - dense_9_accuracy_1: 0.0000e+00 - dense_9_accuracy_2: 0.1000 - dense_9_accuracy_3: 0.3000 - dense_9_accuracy_4: 0.4000 - dense_9_accuracy_5: 0.7000 - dense_9_accuracy_6: 0.7000 - dense_9_accuracy_7: 0.7000 - dense_9_accuracy_8: 0.8000 - dense_9_accuracy_9: 0.8000 - dense_9_accuracy_10: 0.8000 - dense_9_accuracy_11: 0.9000 - dense_9_accuracy_12: 0.9000 - dense_9_accuracy_13: 0.9000 - dense_9_accuracy_14: 0.9000 - dense_9_accuracy_15: 0.9000 - dense_9_accuracy_16: 0.9000 - dense_9_accuracy_17: 0.9000 - dense_9_accuracy_18: 0.9000 - dense_9_accuracy_19: 0.9000                                                       
Epoch 2/10
10/10 [==============================] - 1s 138ms/step - loss: 39.0851 - dense_9_loss: 0.2484 - dense_9_accuracy: 0.0000e+00 - dense_9_accuracy_1: 0.0000e+00 - dense_9_accuracy_2: 0.2000 - dense_9_accuracy_3: 0.4000 - dense_9_accura

In [74]:
import math
def beam_search_decoder(predictions, top_k = 3):

    output_sequences = [([], 0)]
    
    for token_probs in predictions:
        new_sequences = []
        token_probs = token_probs.reshape(20000 ,)
        for old_seq, old_score in output_sequences:
            for char_index in range(len(token_probs)):
                new_seq = old_seq + [char_index]
                new_score = old_score + math.log(token_probs[char_index])
                new_sequences.append((new_seq, new_score))
                
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:top_k]
        
    return output_sequences

In [83]:
s = [("hi")]

In [84]:
seq = tokenizer.texts_to_sequences(s)
pad = pad_sequences(seq,maxlen=max_len, truncating="post",padding = 'post')

In [85]:
pad

array([[236,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0]], dtype=int32)

In [86]:
pred = model.predict([pad , np.zeros((1,128)) , np.zeros((1,128))])


In [87]:
pred

[array([[2.4002235e-01, 6.4382175e-06, 3.9195258e-02, ..., 6.5357767e-06,
         6.2875115e-06, 6.6187172e-06]], dtype=float32),
 array([[5.0401425e-01, 4.7028820e-07, 3.4023881e-02, ..., 4.8395975e-07,
         4.5678064e-07, 5.0095156e-07]], dtype=float32),
 array([[6.2403727e-01, 1.7086424e-07, 2.6303453e-02, ..., 1.7774663e-07,
         1.6766701e-07, 1.8787900e-07]], dtype=float32),
 array([[7.0929402e-01, 8.4550443e-08, 2.0170171e-02, ..., 8.8587825e-08,
         8.4037403e-08, 9.5533025e-08]], dtype=float32),
 array([[7.7059841e-01, 4.7283983e-08, 1.5664149e-02, ..., 4.9762409e-08,
         4.7494677e-08, 5.4538560e-08]], dtype=float32),
 array([[8.0891657e-01, 3.0728561e-08, 1.2881107e-02, ..., 3.2418459e-08,
         3.1046572e-08, 3.5853862e-08]], dtype=float32),
 array([[8.2998258e-01, 2.3320759e-08, 1.1391275e-02, ..., 2.4635296e-08,
         2.3607361e-08, 2.7313760e-08]], dtype=float32),
 array([[8.4102190e-01, 1.9808629e-08, 1.0644097e-02, ..., 2.0942164e-08,
         

In [88]:

ans = beam_search_decoder(pred)

In [89]:
ans

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -5.628117419079653),
 ([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -7.4402936735550265),
 ([4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  -7.476828381036153)]

In [82]:
index_word[4]
# index_word[3]

'i'